Copyright 2021 Google LLC

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

In [ ]:
import sklearn
import numpy as np
from sklearn import svm
from tensorflow.keras.datasets import fashion_mnist

(trn_x, trn_y), (tst_x, tst_y) = fashion_mnist.load_data()
twoclass_inds = np.where(trn_y<=1)[0]
trn_x, trn_y = trn_x[twoclass_inds], trn_y[twoclass_inds]
trn_x = trn_x.reshape((trn_x.shape[0], -1))/255.0 - .5
trn_ct, pois_ct = 2000, 200
print(trn_x.shape, trn_y.shape)

# FMNIST clean

In [ ]:
# linear
print("Total Samples: {}".format(trn_ct))
total_sv = []
for _ in range(5):
  fm_trn_inds = np.random.choice(trn_x.shape[0], trn_ct, replace=False)
  fm_trn_x, fm_trn_y = trn_x[fm_trn_inds], trn_y[fm_trn_inds]
  clean_linear = svm.SVC(kernel='linear')
  clean_linear.fit(fm_trn_x, fm_trn_y)
  duals = np.array(clean_linear.dual_coef_)
  zero_duals = np.isclose(duals, 0).astype(np.int)
  is_sv = (zero_duals.sum(axis=0)<=1).astype(np.int)
  total_sv.append(is_sv.sum())
print("Total Support Vectors: {}".format(np.mean(total_sv)))

In [ ]:
# linear
print("Total Samples: {}".format(trn_ct))
total_sv = []
for _ in range(5):
  fm_trn_inds = np.random.choice(trn_x.shape[0], trn_ct, replace=False)
  fm_trn_x, fm_trn_y = trn_x[fm_trn_inds], trn_y[fm_trn_inds]

  clean_poly = svm.SVC(kernel='poly')
  clean_poly.fit(fm_trn_x, fm_trn_y)
  duals = np.array(clean_poly.dual_coef_)
  zero_duals = np.isclose(duals, 0).astype(np.int)
  is_sv = (zero_duals.sum(axis=0)<=1).astype(np.int)
  total_sv.append(is_sv.sum())
print("Total Support Vectors: {}".format(np.mean(total_sv)))

# FMNIST poison

In [ ]:
# linear

print("Total Samples: {}".format(trn_ct))
total_sv = []
for _ in range(5):
  fm_inds = np.random.choice(trn_x.shape[0], trn_ct+pois_ct, replace=False)
  fm_trn_x, fm_trn_y = trn_x[fm_inds[:trn_ct]], trn_y[fm_inds[:trn_ct]]
  fm_pois_x = trn_x[fm_inds[trn_ct:]]
  fm_pois_y = 1-trn_y[fm_inds[trn_ct:]]
  fm_poised_x = np.concatenate((fm_trn_x, fm_pois_x))
  fm_poised_y = np.concatenate((fm_trn_y, fm_pois_y))
  pois_linear = svm.SVC(kernel='linear')
  pois_linear.fit(fm_poised_x, fm_poised_y)
  trn_support_inds = np.where(pois_linear.support_<trn_ct)[0]
  duals = np.array(pois_linear.dual_coef_[:, trn_support_inds])
  zero_duals = np.isclose(duals, 0).astype(np.int)
  is_sv = (zero_duals.sum(axis=0)<=1).astype(np.int)
  total_sv.append(is_sv.sum())
print("Total Support Vectors: {}".format(np.mean(total_sv)))

In [ ]:
# poly
print("Total Samples: {}".format(fm_trn_x.shape[0]))
total_sv = []
for _ in range(5):
  fm_inds = np.random.choice(trn_x.shape[0], trn_ct+pois_ct, replace=False)
  fm_trn_x, fm_trn_y = trn_x[fm_inds[:trn_ct]], trn_y[fm_inds[:trn_ct]]
  fm_pois_x = trn_x[fm_inds[trn_ct:]]
  fm_pois_y = 1-trn_y[fm_inds[trn_ct:]]
  fm_poised_x = np.concatenate((fm_trn_x, fm_pois_x))
  fm_poised_y = np.concatenate((fm_trn_y, fm_pois_y))

  pois_poly = svm.SVC(kernel='poly')
  pois_poly.fit(fm_poised_x, fm_poised_y)
  trn_support_inds = np.where(pois_poly.support_<fm_trn_x.shape[0])[0]
  duals = np.array(pois_poly.dual_coef_[:, trn_support_inds])
  zero_duals = np.isclose(duals, 0).astype(np.int)
  is_sv = (zero_duals.sum(axis=0)<=1).astype(np.int)
  total_sv.append(is_sv.sum())
print("Total Support Vectors: {}".format(np.mean(total_sv)))

In [ ]:
trn_ct, pois_ct = 1000, 1 
fm_inds = np.random.choice(trn_x.shape[0], trn_ct+1, replace=False)
fm_trn_x, fm_trn_y = trn_x[fm_inds[:trn_ct]], trn_y[fm_inds[:trn_ct]]
targ_x, targ_y = trn_x[fm_inds[trn_ct:trn_ct+1]], trn_y[fm_inds[trn_ct:trn_ct+1]]
pois_x, pois_y = np.concatenate([targ_x.copy() for _ in range(pois_ct)]), np.ones(pois_ct)-targ_y
print(fm_trn_x.shape, fm_trn_y.shape, targ_x.shape, targ_y.shape, pois_x.shape, pois_y.shape)

clean_svm = svm.SVC(kernel='linear')
clean_svm.fit(fm_trn_x, fm_trn_y)

in_x, in_y = np.concatenate([targ_x, fm_trn_x]), np.concatenate([targ_y, fm_trn_y])
in_svm = svm.SVC(kernel='linear')
in_svm.fit(in_x, in_y)
print(in_svm.support_)

poised_x, poised_y = np.concatenate([pois_x, fm_trn_x]), np.concatenate([pois_y, fm_trn_y])
poised_inx, poised_iny = np.concatenate([targ_x, pois_x, fm_trn_x]), np.concatenate([targ_y, pois_y, fm_trn_y])

p_svm = svm.SVC(kernel='linear')
p_svm.fit(poised_x, poised_y)
print(p_svm.support_)

pin_svm = svm.SVC(kernel='linear')
pin_svm.fit(poised_inx, poised_iny)
print(pin_svm.support_)


def poison(model, target_x, target_y):
  pass